<a href="https://colab.research.google.com/github/thanugeorge13/BIS_LAB/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

CHROME_LENGTH = 5
POP_SIZE = 4
CROSS_RATE = 0.8
MUT_RATE = 0.1

def fitness(x):
  return x**2

def encode(x):
  if x >= 0:
    return format(x, f'0{CHROME_LENGTH}b')
  else:
    return bin((1 << CHROME_LENGTH) + x)[2:]


def decode(x):
  if x[0] == '0':
    return int(x, 2)
  else:
    return int(x, 2) - (1 << CHROME_LENGTH)

pop = []
for i in range(POP_SIZE):
  pop.append(random.randint(-10, 10))

#print(INITIAL_POPULATION)

def roulette_selection(pop,fitness):
  total_fitness = sum(fitness)
  pick = random.uniform(0,total_fitness)
  current = 0
  for i in range(len(pop)):
    current += fitness[i]
    if current > pick:
      return pop[i]
  return pop[-1]

def crossover(parent1,parent2):
  if random.random() < CROSS_RATE:
    index = random.randint(1,CHROME_LENGTH-1)
    child1 = parent1[:index] + parent2[index:]
    child2 = parent2[:index] + parent1[index:]
    return child1,child2
  else:
    return parent1,parent2

def mutate(child):
  child = list(child)
  for i in range(len(child)):
    if random.random() < MUT_RATE:
      index = random.randint(0,CHROME_LENGTH-1)
      child[index] = '1' if child[index] == '0' else '0'
  return ''.join(child)

def genetic_algorithm():

    population = [encode(x) for x in pop]
    print("Initial Population:", population, [decode(c) for c in population])

    for gen in range(1, 11):

        decoded = [decode(c) for c in population]
        fitnesses = [fitness(x) for x in decoded]

        total_fit = sum(fitnesses)
        probs = [f/total_fit for f in fitnesses]
        expected = [p*POP_SIZE for p in probs]

        print(f"\nGeneration {gen}")
        for i in range(POP_SIZE):
            print(f"x={decoded[i]}, bin={population[i]}, fit={fitnesses[i]}, "
                  f"prob={probs[i]:.3f}, exp_count={expected[i]:.2f}")

        new_pop = []
        while len(new_pop) < POP_SIZE:

            p1 = roulette_selection(population, fitnesses)
            p2 = roulette_selection(population, fitnesses)
            c1, c2 = crossover(p1, p2)
            c1, c2 = mutate(c1), mutate(c2)
            new_pop.extend([c1, c2])

        population = new_pop[:POP_SIZE]

    decoded = [decode(c) for c in population]
    fitnesses = [fitness(x) for x in decoded]
    best_idx = fitnesses.index(max(fitnesses))
    print("\nFinal Best Solution:", decoded[best_idx], population[best_idx], "fitness=", fitnesses[best_idx])

genetic_algorithm()

Initial Population: ['00011', '11111', '11000', '11101'] [3, -1, -8, -3]

Generation 1
x=3, bin=00011, fit=9, prob=0.108, exp_count=0.43
x=-1, bin=11111, fit=1, prob=0.012, exp_count=0.05
x=-8, bin=11000, fit=64, prob=0.771, exp_count=3.08
x=-3, bin=11101, fit=9, prob=0.108, exp_count=0.43

Generation 2
x=-5, bin=11011, fit=25, prob=0.212, exp_count=0.85
x=-8, bin=11000, fit=64, prob=0.542, exp_count=2.17
x=-5, bin=11011, fit=25, prob=0.212, exp_count=0.85
x=2, bin=00010, fit=4, prob=0.034, exp_count=0.14

Generation 3
x=-8, bin=11000, fit=64, prob=0.360, exp_count=1.44
x=-8, bin=11000, fit=64, prob=0.360, exp_count=1.44
x=-5, bin=11011, fit=25, prob=0.140, exp_count=0.56
x=-5, bin=11011, fit=25, prob=0.140, exp_count=0.56

Generation 4
x=-8, bin=11000, fit=64, prob=0.416, exp_count=1.66
x=-1, bin=11111, fit=1, prob=0.006, exp_count=0.03
x=-5, bin=11011, fit=25, prob=0.162, exp_count=0.65
x=-8, bin=11000, fit=64, prob=0.416, exp_count=1.66

Generation 5
x=8, bin=01000, fit=64, prob=0.3